# CPF enricher
O enricher de cpf utiliza a api da serpro para buscar os dados de abrigados e enriquecer o documento na collection abrigado.

In [ ]:
%load_ext blackcellmagic

## Instalação de dependências

In [ ]:
pip install -r ../requirements.txt

In [2]:
## Importação de dependências
import os
import re
import base64
import requests
import pandas as pd
import firebase_admin
import uuid
import json
import sys
import math
from firebase_admin import credentials
from firebase_admin import firestore
from dataclasses import dataclass, field
from typing import Optional, List, Dict
from enum import Enum
from datetime import datetime, timezone

sys.path.append("../../modules/environment_selector")
from selector import environment_selector

sys.path.append("../../modules/firebase_manager")
from firestore_query_batch import query_batch
from firestore_model_abrigo_legacy import AbrigoLegacyEntity
from firestore_upload_data import upload_to_firestore

sys.path.append("../../modules/data_type_handler")
from nome import normalize_nome
from cpf import extract_cpf 

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Test: function should match the CPF in the string


## Configuração

### Environment Selector

In [ ]:
environment_selector()

### Configura chaves e parâmetros

In [ ]:
# Firebase Firestore Config
base_path = os.getenv("SOSRS_BASEPATH")
sosrs_environment = os.getenv("SOSRS_ENVIRONMENT")
sosrs_firestore_keyfile = os.getenv("SOSRS_FIRESTORE_KEYFILE")
sosrs_firestore_path = os.path.join(base_path, sosrs_firestore_keyfile)

# CPF Enricher Service Config
serpro_customer_key = os.getenv("SERPRO_CPF_CUSTOMER_KEY")
serpro_customer_secret = os.getenv("SERPRO_CPF_CONSUMER_SECRET")
serpro_development_bearer_token = os.getenv("SERPRO_CPF_BEARER_TOKEN")
serpro_service_url = os.getenv("SERPRO_CPF_SERVICE_URL")
serpro_service_auth_url = os.getenv("SERPRO_CPF_AUTH_URL")
serpro_service_token_ttl_secs = os.getenv("SERPRO_CPF_TOKEN_TTL_SECS")
serpro_path_get_cpf = os.getenv("SERPRO_CPF_PATH_GET_CPF")
serpro_path_get_cpf_status = os.getenv("SERPRO_CPF_PATH_GET_CPF_STATUS")

### Autenticação do Serpro
Autenticação com serviço de consulta de CPF.
Dado a característica da API, que utiliza o mesmo endpoint para ambos os ambientes, modificando apenas o path.

In [ ]:
# Define header condicional para suporte ambiente de desenvolvimento e produção
bear_token = None

if sosrs_environment == "development":
    bear_token = serpro_development_bearer_token
else:
    credentials = (f"{serpro_customer_key}:{serpro_customer_secret}").encode("utf-8")
    credentials_base64 = base64.b64encode(credentials)
    credentials_base64_encoded = credentials_base64.decode("utf-8")

    headers = {"Authorization": f"Basic {credentials_base64_encoded}", "Content-Type": "application/x-www-form-urlencoded"}

    body = {"grant_type": "client_credentials"}

    response = requests.post(serpro_service_auth_url, headers=headers, data=body)
    if response.status_code == 200:
        bear_token = response.json()["access_token"]
        print("Token obtido com sucesso.")
    else:
        print("Falha ao obter token:", response.status_code, response.text)

print("Token de acesso:", bear_token)

#### busca dados de um cpf

In [ ]:

cred = credentials.Certificate(sosrs_firestore_path)
firebase_admin.initialize_app(cred)
client_firestore = firestore.client()

In [ ]:
# Cria objetos para manipular retorno da API Serpro
class CpfSituacao(Enum):
    REGULAR = ("0", "Regular")
    SUSPENSA = ("2", "Suspensa")
    TITULAR_FALECIDO = ("3", "Titular Falecido")
    PENDENTE_REGULARIZACAO = ("4", "Pendente de Regularização")
    CANCELADA_MULTIPLICIDADE = ("5", "Cancelada por Multiplicidade")
    NULA = ("8", "Nula")
    CANCELADA_OFICIO = ("9", "Cancelada de Ofício")

    def __init__(self, codigo, descricao):
        self.codigo = codigo
        self.descricao = descricao

    @staticmethod
    def from_codigo(codigo):
        for situacao in CpfSituacao:
            if situacao.codigo == codigo:
                return situacao
        raise ValueError(f'Codigo de situação do cpf desconhecido: {codigo}. Necessário revisar documentação da API.')
        
@dataclass
class Pessoa:
    ni: str
    nome: str
    nascimento: str
    situacao: CpfSituacao

In [ ]:
document_ref = client_firestore.collection("abrigado").document("0Be8ybUyY9fNRZqvrHBO")
document = document_ref.get()
abrigado = document.to_dict()
print(abrigado)

cpf = cpf = abrigado["responsavel"]["documentos"]["cpf"]
print(cpf)

headers = {"Authorization": f"Bearer {bear_token}"}

# Constroi url
# Interpolar o CPF no caminho
path_with_cpf = serpro_path_get_cpf.format(num_cpf=cpf)
url = f"{serpro_service_url}{path_with_cpf}"

response = requests.get(url, headers=headers)
data = None
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    None
    
pessoa = Pessoa(
    ni=data['ni'],
    nome=data['nome'],
    nascimento=data['nascimento'],
    situacao=CpfSituacao.from_codigo(data['situacao']['codigo'])  # Passando o objeto Enum
)

print(pessoa)

## Enriquecimento de dados abrigados

##### Códigos de erro

### Verifica o status do CPF
O objetivo é fazer um análise do CPF em um status fraudulento, como por exemplo, o CPF de uma pessoa falecida.

In [ ]:
# # Definir o nome da coleção e o arquivo de saída
# collection_name = "abrigado"
# output_file = "../temp/abrigado_export.xlsx"


# def datetime_converter(o):
#     if isinstance(o, datetime):
#         return o.__str__()


# def load_collection(collection_name):
#     try:
#         collection_ref = client_firestore.collection(collection_name)
#         docs = collection_ref.stream()

#         all_docs = []
#         for doc in docs:
#             try:
#                 doc_dict = doc.to_dict()
#                 for key, value in doc_dict.items():
#                     if isinstance(value, datetime):
#                         doc_dict[key] = value.isoformat()
#                 all_docs.append(
#                     doc_dict
#                 )  # Certifique-se que esta linha está dentro do loop
#             except Exception as e:
#                 print(f"Erro ao processar documento {doc.id}: {e}")

#         if not all_docs:  # Checa se a lista está vazia
#             print("Nenhum documento foi carregado da coleção.")
#         return all_docs

#     except Exception as e:
#         print(f"Erro ao carregar coleção: {e}")
#         return []


# def export_to_excel(data, output_file):
#     try:
#         # Criando um DataFrame a partir dos dados
#         df = pd.DataFrame(data)

#         if df.empty:  # Verifica se o DataFrame está vazio
#             print("O DataFrame está vazio. Nenhum dado para exportar.")
#         else:
#             # Salvando o DataFrame em um arquivo Excel
#             df.to_excel(output_file, index=False)
#             print(f"Dados exportados para {output_file}")

#     except Exception as e:
#         print(f"Erro ao salvar arquivo Excel: {e}")


# # Carregando dados da coleção
# data = load_collection(collection_name)

# # Exportando dados para o arquivo Excel
# export_to_excel(data, output_file)